# Personal Assistant for Children with Special Needs

## Overview

This Jupyter notebook documents the creation of a "Virtual Friend" personal assistant application. It is designed specifically for children  with Attention Deficit Hyperactivity Disorder (ADHD). Using advanced technologies like Large Language Models (LLM) and Natural Language Processing (NLP), this tool is aim to enhance communication skills and maintain engagement in everyday social interactions through interactive support.


### 1. Environment Setup and Imports

In [1]:
# !pip install python-dotenv
# !pip install openai==1.23.2
# !pip install flask_cors

In [1]:
import openai
import time
print(openai.__version__)
# https://platform.openai.com/docs/assistants/overview?context=with-streaming

1.23.2


### 2. API Client Initialization and Assistant Configuration

Initiate the OpenAI API client and set up the virtual assistant name and ID for future interactions.


In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv

'''
I didn't want to expose my api keys that's why I used dotenv library
.env file needs to be added with the content

OPENAI_API_KEY="XXXXXX"
''' 
load_dotenv()

client = OpenAI(
     api_key = os.environ.get("OPENAI_API_KEY")
)

assistant_name = "Virtual Friend"
assistant_id = "asst_waiBYAHIZoJqFZWa4mzJ9ivX"

### 3. Create the Personal Assistant

The function is crucial for setting up the assistant with specific settings tailored to the needs of children with ADHD, using the capabilities of OpenAI's GPT-4 Turbo model.


In [3]:
def create_assistant():
    assistant = client.beta.assistants.create(
      name="Virtual Friend",
      instructions="""You are a supportive companion for an 8-year-old child with ADHD. 
      Engage in conversation with him to help maintain focus on the same topic.
      If he strays from the subject, redirect his attention back to the topic in an engaging manner.
      """,
      tools=[{"type": "code_interpreter"}],
      model="gpt-4-turbo",
    )
    return assistant

### 4. Retrieve the Personal Assistant

This function retrieves an existing virtual assistant using its ID or creates a new one if it doesn't exist.So we can ensure the assistant is always available for user.

In [4]:
def get_assistant():
    if assistant_id != None:
        return client.beta.assistants.retrieve(assistant_id=assistant_id)
    
    for assistant in client.beta.assistants.list().data:
        print(assistant.name, assistant.id)
    
        if assistant.name == assistant_name:
            # return assistant.id
            print(assistant.id)
            return client.beta.assistants.retrieve(assistant_id=assistant.id)

    assistant = create_assistant()
    print (assistant.id)
    return assistant

In [5]:
assistant = get_assistant()

### 5. Handling Tokens
This function is used to collect the token and later sent to the client word by word

In [6]:
tokens = []

def stream(token):
    global tokens
    tokens.append(token)
        

### 6. Create a EventHandler Class 

First, create an EventHandler class to define how to handle events in the response stream.


In [7]:
from typing_extensions import override
from openai import AssistantEventHandler

 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    # print(f"\nassistant > ", end="", flush=True)
    pass
      
  @override
  def on_text_delta(self, delta, snapshot):
    # print(delta.value, end="", flush=True)
    stream(delta.value)
    self.on_token(delta.value)

  def on_token(self, token):
      pass

  def on_end(self):
    # print('ended')
    stream(None)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)
 

### 7. Create a Thread
A Thread represents a conversation between a user and one or more Assistants. You can create a Thread whenever a user, or your AI application, initiates a conversation with your Assistant.

In [8]:
# moved into the start function
#thread = client.beta.threads.create()

### 8. Adding Message to the Thread and Create a Run
Initiate a conversation by adding messages to the thread as the user asks questions.

Note: Intentionally, I inject a prompt "Hi" to initiate a conversation but hide it from the user so it appears like initiated the conversation 

In [9]:
def start():
    thread = client.beta.threads.create()
    
    first = True
    while True:
        if first == True:
            first = False
            user_input = 'Hi'
        else:
            user_input = input()

        if user_input == "":
            print("--- exiting")
            break
 
        message = client.beta.threads.messages.create(
          thread_id=thread.id,
          role="user",
          content=user_input
        )
        
        
        with client.beta.threads.runs.stream(
          thread_id=thread.id,
          assistant_id=assistant.id,
          event_handler=EventHandler(),
        ) as stream:
          stream.on_token = lambda word: print(word, end="", flush=True)
          stream.until_done()

In [10]:
# This is the actual dialog between 8-years-old boy and the Personal Assistant on 5/2/2024

In [11]:
# start()

### 9. Generating Responses for Web Application

In [12]:

def generate(): 
    global tokens
    while True:
      if len(tokens) > 0:
        item = tokens.pop(0)
        if item == None: 
            break
        yield item
        time.sleep(0.1)
    
def prompt(user_input, thread):
    message = client.beta.threads.messages.create(
          thread_id=thread.id,
          role="user",
          content=user_input
        )

    
     
    with client.beta.threads.runs.stream(
      thread_id=thread.id,
      assistant_id=assistant.id,
      event_handler=EventHandler(),
    ) as stream:
      # stream.on_token = sentence 

      stream.until_done()

    return None

# thread = client.beta.threads.create()
# prompt('how are you', thread)
# for i in generate():
#     print(i, end="", flush=True)

### 10. Flask Web Application Setup
I've included this step to demonstrate how the personal assistant operates in a web-based environment for the presentation.

In [ ]:
from flask import Flask, render_template, request, redirect, session, Response, jsonify
from flask_cors import CORS, cross_origin
app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'

thread = None 

def reset_thread():
    global thread
    thread = client.beta.threads.create()

reset_thread()

@app.route('/reset', methods=['GET'])
def reset():
    reset_thread()
    return jsonify(True)

@app.route('/query', methods=['GET'])
def handle_get():
    query = request.args['query']
    # print(query)
    prompt(query, thread)
     
    return Response(generate(), mimetype='text/event-stream')
 
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [15/May/2024 19:48:49] "GET /reset HTTP/1.1" 200 -
127.0.0.1 - - [15/May/2024 19:48:55] "GET /query?query=hi HTTP/1.1" 200 -
127.0.0.1 - - [15/May/2024 19:48:57] "GET /query?query=hi HTTP/1.1" 200 -
127.0.0.1 - - [15/May/2024 19:50:29] "GET /query?query=and%20right%20till%20then%20you%20can%20focus%20on%20that HTTP/1.1" 200 -


#### SUMMARY 

In this project, I developed a virtual personal assistant to help children with Attention-Deficit Hyperactivity Disorder (ADHD). Using OpenAI's Assistant API, the assistant aims to enhance verbal communication skills and keep children engaged by sustaining their focus during interactions. 

The assistant is instructed to maintain engaging conversations that align with children's interests while subtly steering them to stay on topic. The assistant enables real-time understanding and reaction to the child's input. 

In testing, the personal assistant demonstrates understanding and responds appropriately, even to inputs that include emojis. Its ability to ask follow-up questions, provide enlightening insights, and maintain specific contexts conversationally shows its adaptability and customization potential.


However, designing this tool for children with ADHD presents several challenges, especially in providing precise instructions to foster meaningful dialogues between the child and the assistant. It's crucial to give strategic instructions that engage the child's interests while gently guiding them to stay focused on the current topic.


